In [2]:
import os 
import os.path
import numpy as np
import copy

# clean content
def clean_content(imdb_str):
    
    imdb_word_split = []
    count_flag = 0
    imdb_str = imdb_str.lower() # 大小写转换
    for i in range(len(imdb_str) - 1):

        if(imdb_str[i] is ' ')and(imdb_str[i+1] is not ' ')and(count_flag is 0):
            start_index = i+1
            count_flag = 1

        if(imdb_str[i] is not ' ')and(imdb_str[i+1] is ' ')and(count_flag is 1):
            end_index = i+1
            imdb_word_split.append(imdb_str[start_index:end_index])
            count_flag = 0
    return imdb_word_split


subj_file_name = 'quote.tok.gt9.5000'  # 主观
subj_file  = []
subj_label = []
with open(subj_file_name,'r') as f:  #注意open 和 codecs.opem
    lines = f.readlines()
    for i in range(len(lines)):
        lines_clean = lines[i].rstrip('\n')
        subj_file.append(' ' + lines_clean)
        
        
obj_file_name  = 'plot.tok.gt9.5000'   # 客观
obj_file  = []
obj_label = []

with open(obj_file_name,'r') as f:  #注意open 和 codecs.opem
    lines = f.readlines()
    for i in range(len(lines)):
        lines_clean = lines[i].rstrip('\n')
        obj_file.append(' ' + lines_clean)    
        
train_file = subj_file[0:4500] + obj_file[0:4500]
test_file  = subj_file[4500:5000] + obj_file[4500:5000]
    
sub_train_label = np.concatenate([np.ones(4500),np.zeros(4500)], axis = 0)
sub_train_split = []
for i in range(len(sub_train_label)):
    sub_train_split.append(clean_content(train_file[i]))
    
sub_test_label = np.concatenate([np.ones(500),np.zeros(500)], axis = 0)
sub_test_split = []
for i in range(len(sub_test_label)):
    sub_test_split.append(clean_content(test_file[i]))    

sub_train_list = train_file
sub_test_list  = test_file

# calculate vab
sub_vab_count = {}

for i in range(len(sub_train_split)):
    sub_str = sub_train_split[i]
    for word_index in range(len(sub_str)):
        if (not sub_vab_count.has_key(sub_str[word_index])):
            sub_vab_count.update({sub_str[word_index]: 1})
        else:
            sub_vab_count[sub_str[word_index]] = sub_vab_count[sub_str[word_index]] + 1 
                
del_stop_word = False
if del_stop_word is True:
    # stop word list
    stop_word_list = []
    with open('/home/xiaosucheng/Desktop/IMDB/IMDB_Read_Data/imdb_stop_word.txt','r') as f:  #注意open 和 codecs.opem
        line = f.readlines()
        for i in range(len(line)):
            stop_word_list.append(line[i].rstrip('\n'))

    for i in range(len(stop_word_list)):
        if sub_vab_count.has_key(stop_word_list[i]):
            sub_vab_count.pop(stop_word_list[i])
    print 'delete stop word' 

    
# sort vab
sub_vab                  = {}
sub_vab_list_orgin       = sub_vab_count.keys()  # list
sub_vab_count_list_orgin = sub_vab_count.values() # list
sub_vab_list             = []
sub_vab_count_list       = []

sub_vab_index            = np.argsort(-np.array(sub_vab_count_list_orgin))  # np.argsort(-x) 降序排序
for i in range(len(sub_vab_list_orgin)):
    sub_vab.update({sub_vab_list_orgin[sub_vab_index[i]]: i + 1})    # 0 给stop_word　所以+1
    sub_vab_list.append(sub_vab_list_orgin[sub_vab_index[i]])
    sub_vab_count_list.append(sub_vab_count_list_orgin[sub_vab_index[i]])

print 'calculate vocabulary finished'

vab_size = 3000
# word2index 如果不含有该词或者index超过vab_size,则置成０

sub_train_list_index = copy.deepcopy(sub_train_split)

for i in range(len(sub_train_split)):

    sub_str = sub_train_split[i]
    for word_index in range(len(sub_str)):
        if (not sub_vab.has_key(sub_str[word_index])) or (sub_vab[sub_str[word_index]] >= vab_size):
            sub_train_list_index[i][word_index] = 0
        else:
            sub_train_list_index[i][word_index] = sub_vab[sub_str[word_index]]

print 'train word2index finished'

sub_test_list_index = copy.deepcopy(sub_test_split)
for i in range(len(sub_test_split)):
    
    sub_str = sub_test_split[i]
    for word_index in range(len(sub_str)):
        if (not sub_vab.has_key(sub_str[word_index])) or (sub_vab[sub_str[word_index]] >= vab_size):
            sub_test_list_index[i][word_index] = 0
        else:
            sub_test_list_index[i][word_index] = sub_vab[sub_str[word_index]]

print 'test word2index finished'

# imdb_vab_list 　　　　　 词表的词
# imdb_vab_count_list 　 词表词的词频
# imdb_list_index 　　　 每个文档每个词的index'
# 对应index　０　表示不在表内的词
# 找到词SUB['Vocabulary'][index-1] 这里注意减１
import cPickle
SUB = {}
SUB['Vocabulary']       = sub_vab_list
SUB['Vab_count']        = sub_vab_count_list
SUB['Vab_Size']         = vab_size
SUB['Label']            = ['Subject', 'Object']
SUB['Train_Origin']     = sub_train_list
SUB['Train_Label']      = sub_train_label
SUB['Train_Word_Split'] = sub_train_split
SUB['Train_Word2Index'] = sub_train_list_index
SUB['Test_Origin']      = sub_test_list
SUB['Test_Label']       = sub_test_label
SUB['Test_Word_Split']  = sub_test_split
SUB['Test_Word2Index']  = sub_test_list_index
cPickle.dump(SUB, open('./SUB_20k.pkl','wb'))
print 'save finished'   

calculate vocabulary finished
train word2index finished
test word2index finished
save finished


In [2]:
import numpy as np
import scipy.io as sio

# ELEC['Train_BOW'] = np.zeros([30000, 25000])
# for i in range(len(ELEC['Train_Word2Index'])):
    
#     elec_str = ELEC['Train_Word2Index'][i]
#     for j in range(len(elec_str)):
        
#         ELEC['Train_BOW'][elec_str[j],i] = ELEC['Train_BOW'][elec_str[j],i] + 1
        
# ELEC['Test_BOW'] = np.zeros([30000, 25000])
# for i in range(len(ELEC['Test_Word2Index'])):
    
#     elec_str = ELEC['Test_Word2Index'][i]
#     for j in range(len(elec_str)):
        
#         ELEC['Test_BOW'][elec_str[j],i] = ELEC['Test_BOW'][elec_str[j],i] + 1        
        
    
sio.savemat('SUBJ_ALL_5k.mat',{'SUBJ_Train_Word2Index': SUB['Train_Word2Index'],
                               'SUBJ_Vab': SUB['Vocabulary'],
                               'SUBJ_Test_Word2Index':SUB['Test_Word2Index'],
                               'SUBJ_Train_Label': list(SUB['Train_Label']),
                               'SUBJ_Test_Label': list(SUB['Test_Label'])
                               })

print 'save mat finished'

save mat finished
